In [ ]:
pip install requests beautifulsoup4

In [12]:
import requests
from bs4 import BeautifulSoup

In [9]:
url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
paragraphs = soup.find_all()
with open('webpage_content.txt', 'w', encoding='utf-8') as file:
    for paragraph in paragraphs:
        file.write(paragraph.text + '\n')

scrape for the urls:

In [34]:
main_url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy'
response = requests.get(main_url)

# Parse the HTML content and Find all <a> tags
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a')

# Extract the href attribute from each <a> tag and filter only relevant ones 
nested_urls = [link.get('href') for link in links if link.get('href')]
filtered_urls = [url for url in nested_urls if url.startswith(main_url + "/")]

nested_nested_urls = []
for secondary_url in filtered_urls:
    response = requests.get(secondary_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')

    temp = [link.get('href') for link in links if link.get('href')]
    nested_nested_urls += temp



In [62]:
# filter all the nested_nested_urls:
filtered_urls = [url for url in nested_nested_urls if url.startswith(main_url + "/")]

filtered_urls2 = [filtered_urls[0]]

for i in range(1, len(filtered_urls)):
    # Compare each element with the previous one
    if filtered_urls[i] != filtered_urls[i-1]:
        # Add it to the filtered list if it's different
        filtered_urls2.append(filtered_urls[i])


for all the content:

In [59]:
url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy/session/60/contribution/4'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
paragraphs = soup.find_all('p')

with open('webpage_content.txt', 'w', encoding='utf-8') as file:
    for paragraph in paragraphs:
        file.write(paragraph.text + '\n')

        


scaling the above to all

In [68]:
i=1

with open('all_text2.txt', 'w') as file:
    pass  # This clears the file

def append_to_file(content, filename='all_text2.txt'):
    with open(filename, 'a', encoding='utf-8') as file:
        file.write(content + '\n')

for url in filtered_urls2:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')

    append_to_file("abstract " + str(i))  
    for paragraph in paragraphs:
        append_to_file(paragraph.text)
    i+=1 

In [70]:
# cleaning:

unwanted_phrases = [
    "Auswahlstatus für diesen Beitrag: ",
    "Titel und präsentierender Autor",
    "Titel und Autorenblock",
    "Titel, Autorenblock und Kurzfassung",
    "gemäß den Sitzungseinstellungen"
]

def clean_file(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as infile, \
         open(output_filename, 'w', encoding='utf-8') as outfile:
        for line in infile:
            # Check if the line contains any of the unwanted phrases
            if not any(unwanted_phrase in line for unwanted_phrase in unwanted_phrases):
                outfile.write(line)

# Replace 'your_input_file.txt' with the path to your input file
# and 'cleaned_file.txt' with the desired output file path
clean_file('all_text2.txt', 'cleaned_file.txt')

Make pretier:

In [ ]:
def format_entry(entry):
    # Split the entry by lines
    lines = entry.split('\n')
    # Extract and format title, authors, and affiliation
    title_authors_affiliation = lines[0].split('—')
    title = title_authors_affiliation[0].strip()
    authors_affiliation = '—'.join(title_authors_affiliation[1:]).strip()
    authors, affiliation = authors_affiliation.rsplit('—', 1)
    # Extract and format abstract
    abstract = ' '.join(lines[1:]).strip()
    return f"Title: {title}\nAuthors: {authors}\nAffiliation: {affiliation}\nAbstract: {abstract}\n"

def process_document(document_content):
    entries = document_content.split('abstract ')[1:]  # Splitting document into entries, ignoring the very first split part before 'abstract '
    formatted_entries = [format_entry(entry) for entry in entries]
    return "\n".join(formatted_entries)

Tests:

In [29]:
a = [1,2]
b = [4, 3]
a += b
print(c)
print(a)

[1, 2, 4, 3]
[1, 2, 4, 3]


In [63]:
# listing all the urls that make sence  
with open('urls2.txt', 'w', encoding='utf-8') as file:
    for link in filtered_urls2:
        file.write(link + '\n')